In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 8.5 MB/s eta 0:00:00


#Section 1. - Importing and preprocessing the data

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix

# Loading the dataset
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv"
df = pd.read_csv(url)

# Converting SMILES to RDKit Mol objects
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)

# Calculating RDKit descriptors
def calculate_descriptors(mol):
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol)
    }
    return pd.Series(descriptors)

df = df.join(df['mol'].apply(calculate_descriptors))

# Dropping rows with missing values (where molecules couldn't be processed)
df = df.dropna()

# Defining features (X) and target (y)
X = df[['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'Number of Rings', 'Number of Rotatable Bonds']]
y = df['measured log solubility in mols per litre']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
df

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles,mol,MolWt,LogP,NumHDonors,NumHAcceptors
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.770,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e0c120>,457.432,-3.10802,7.0,12.0
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.300,Cc1occc1C(=O)Nc2ccccc2,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e0c190>,201.225,2.84032,1.0,2.0
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.060,CC(C)=CCCC(C)=CC(=O),<rdkit.Chem.rdchem.Mol object at 0x7d4e95e0c200>,152.237,2.87800,0.0,1.0
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.870,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e0c270>,278.354,6.29940,0.0,0.0
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.330,c1ccsc1,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e0c2e0>,84.143,1.74810,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,halothane,-2.608,1,197.381,0,0,0,0.00,-1.710,FC(F)(F)C(Cl)Br,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e43060>,197.381,2.50850,0.0,0.0
1124,Oxamyl,-0.908,1,219.266,1,0,1,71.00,0.106,CNC(=O)ON=C(SC)C(=O)N(C)C,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e430d0>,219.266,0.10710,1.0,5.0
1125,Thiometon,-3.323,1,246.359,0,0,7,18.46,-3.091,CCSCCSP(=S)(OC)OC,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e43140>,246.359,2.99000,0.0,5.0
1126,2-Methylbutane,-2.245,1,72.151,0,0,1,0.00,-3.180,CCC(C)C,<rdkit.Chem.rdchem.Mol object at 0x7d4e95e431b0>,72.151,2.05240,0.0,0.0


#Section 2 - Feeding ML algorithms with data

##Section 2.1 - Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(X_train, y_train)

# Making predictions
y_pred = linear.predict(X_test)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')

RMSE: 1.1331292281411611
R² Score: 0.7602757908887983


##Section 2.2 - Random Forest Regression

In [6]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=None)
forest.fit(X_train, y_train)

# Making predictions
y_pred = forest.predict(X_test)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')

RMSE: 0.6721210042125221
R² Score: 0.8578064424071132


##Section 2.3 - Support Vector Machines

In [7]:
from sklearn.svm import SVR

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the SVR model with RBF kernel
svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Train the model
svr.fit(X_train_scaled, y_train)

# Make predictions
y_pred = svr.predict(X_test_scaled)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')

RMSE: 0.7919337824685895
R² Score: 0.8324589155800377


##Section 2.4 - Multi Layer Perceotron

In [8]:
# Scaling the data (important for MLPs)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining and train the MLPRegressor model
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Making predictions
y_pred = mlp.predict(X_test_scaled)

# Evaluating the model
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')

RMSE: 0.5442793313576517
R² Score: 0.8848525578505047


#Section 3 - Predictions based on SMILES format

In [11]:
from sklearn.preprocessing import StandardScaler
from rdkit.Chem import rdMolDescriptors

# Calculating descriptors for molecules in SMILE format
def calculate_descriptors(mol):
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumRings': rdMolDescriptors.CalcNumRings(mol),
        'NumRotatableBonds': rdMolDescriptors.CalcNumRotatableBonds(mol)
    }
    return pd.Series(descriptors)

# Normalizng the data via StandardScaler
def normalize_data(dataframe):
    scaler = StandardScaler()
    normalized = scaler.fit_transform(dataframe)

    return dataframe

SECTION XX: Testing the ML model on data generated by using rdkit


In [12]:
smilesList = [
    'C', 'CC', 'CCC', 'CCCC', 'CCCCC', 'CCCCCC',
    'CCCCCCC', 'CCN', 'CC(=O)O', 'CCOCC', 'CC(=O)CC',
    'CC(=O)OCC', 'CCCCO', 'C(C(=O)O)C(N)C', 'CCCCCO'
]

# The solubities in a form of log([mol of substance]/[litre of H2O]). Experimental measurements based on the literature.
log_solubilities = [-2.82, -2.70,	-2.82,	-2.98,	-2.65,	-3.96,	-4.53,	-1.57,	1.00,	-0.12,	0.44,	-0.10,	-0.01,	0.79,	-0.61]
# For refernece - "Yalkowsky, S.H., He, Yan, Jain, P. Handbook of Aqueous Solubility Data Second Edition. CRC Press, Boca Raton, FL 2010"


In [13]:
# Creat a SMILES dataframe
smilesDF = pd.DataFrame(smilesList, columns=['smiles'])

# Convert SMILES to RDKit Mol objects
smilesDF['mol'] = smilesDF['smiles'].apply(Chem.MolFromSmiles)

# Calculate the descriptors in a seperate dataframe
descriptors_df = smilesDF['mol'].apply(calculate_descriptors)

# Merge the descripotrs and mol dataframe
testDFfromSMILES = pd.concat([smilesDF.drop(columns=['mol']), descriptors_df], axis=1)

norm_testDFfromSMILES = normalize_data(testDFfromSMILES[['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRings', 'NumRotatableBonds']])

In [14]:
predictions_MLP = svr.predict(norm_testDFfromSMILES)
predictionsDF = norm_testDFfromSMILES.join(pd.DataFrame(predictions_MLP, columns=['MLP predictions']))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


In [15]:
testDFfromSMILES

,smiles,MolWt,LogP,NumHDonors,NumHAcceptors,NumRings,NumRotatableBonds
0,C,16.043,0.6361,0.0,0.0,0.0,0.0
1,CC,30.070,1.0262,0.0,0.0,0.0,0.0
2,CCC,44.097,1.4163,0.0,0.0,0.0,0.0
3,CCCC,58.124,1.8064,0.0,0.0,0.0,1.0
4,CCCCC,72.151,2.1965,0.0,0.0,0.0,2.0
5,CCCCCC,86.178,2.5866,0.0,0.0,0.0,3.0
6,CCCCCCC,100.205,2.9767,0.0,0.0,0.0,4.0
7,CCN,45.085,-0.0350,1.0,1.0,0.0,0.0
8,CC(=O)O,60.052,0.0909,1.0,1.0,0.0,0.0
9,CCOCC,74.123,1.0428,0.0,1.0,0.0,2.0


END
    